In [1]:
%pylab inline --no-import-all
plt.style.use('ggplot')
matplotlib.rc('image', origin='lower', cmap='magma')
matplotlib.rc('axes', grid=False)
from astropy.io import fits
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


# What's in one of these headers, anyway?

In [2]:
fits.getheader('./data/laplace/dd1/contemp_flats_repaired/n47f04p8q_clc_calf.fits')

SIMPLE  =                    T /image conforms to FITS standard                 
BITPIX  =                    8 /bits per data value                             
NAXIS   =                    0 /number of axes                                  
EXTEND  =                    T /file may contain extensions                     
ORIGIN  = 'NOAO-IRAF FITS Image Kernel July 2003' / FITS file originator        
IRAF-TLM= '13:19:18 (26/06/2009)' / Time of last modification                   
NEXTEND =                    5 / Number of standard extensions                  
DATE    = '2009-06-26T19:19:18' / date this file was written (yyyy-mm-dd)       
FILENAME= 'n47f04p8q_clc_cal.fits' / name of file                               
FILETYPE= 'SCI      '          / type of data found in data file                
                                                                                
TELESCOP= 'HST'                / telescope used to acquire data                 
INSTRUME= 'NICMOS'          

Columns it might make sense to have:

  - `ORIENTAT` - position angle of image y axis (deg. e of n)
  - `APERTURE` - aperture in use (NICi,NICi-FIX,NIC2-CORON/ACQ) 
  - `FILTER` - filter used
  - `PA_V3` - position angle of V3-axis of HST (deg)
  - `TARGTYPE`
    - `POINT_SOURCE` - Unresolved, Isolated, Point Source (Star)
    - `EXTENDED_SOURCE` - Spatially Resolved Bright Source (e.g., circumstellar disk)
    - `DOUBLE_STAR` - Double Star With Well Separated Components
    - `CORON_DOUBLE` - ≤ 0.3" overlapping PSF cores inside coronagraph
    - `FAINT_OBJ` - Spatially Resolved Faint Source (e.g., QSO, AGN)
    - `NONE` - No Target or PSF Wings/Halo Appear in the Image
    - `FAILED` - Failed Observation (degraded or useless data)
  - `TARGCNTR`
    - `CENTERED` - Target Nominally Positioned in Coronagraph
    - `DECENTERED` - Target in Coronagraph But Decentered
    - `PERIPHERY` - Target at the Periphery of the Coronagraphic Mask
    - `UNOCCULTED` - Target in FOV but Not Coronagraphically Occulted
    - `UNSTABLE` - Target drifting in aperture
    - `OUTFOV` - Target Out of FOV but PSF Wings (Halo) Seen in FOV
    - `NOTARGET` - No Obvious Target Appears in the Image
  - `PSFQUAL`
    - `OK` - PSF Quality OK for POINT SOURCE Template Subtraction / Image/PSF Quality OK for Science Targets of Interest
    - `BAD` - Target Visible but Pointing Unstable (drifting, etc.)
  - `PSFCRSAT` - `YES` or `NO` - PSF Core Saturated (even just 1 pixel, wings may be OK)
  - `PSFDEGRD` - `YES` or `NO` - PSF Optically Degraded (focus, image smear, other issues)


Take inspiration from ALICE to choose where to concentrate our efforts...

> The PSF libraries assembled by the ALICE pipeline are thus much larger in the F110W and F160W filters than in the other filters. Furthermore, as ALICE’s reprocessing is based on Multi-Reference star Differential Imaging (MRDI) and excludes images with detected circumstellar material from the reference PSF libraries (except in “planet mode”, which includes images of the science target acquired in a different orientation of the spacecraft, when available, Choquet et al. 2014), the medium- and narrow-band filter data were reprocessed with PSF libraries of very small sizes (see Table 1). ALICE’s reprocessing for these datasets is not expected to much improve upon classical PSF subtraction techniques.

— [Hagan et al. in prep](https://archive.stsci.edu/missions/hlsp/alice/hlsp_alice_hst_nicmos_all_multi_v1_readme.pdf)

In [3]:
import glob
from os.path import basename

In [4]:
header_keywords = ('APERTURE', 'FILTER', 'TARGNAME', 'RA_TARG', 'DEC_TARG', 'PA_V3', 'TARGTYPE', 'TARGCNTR', 'PSFQUAL', 'PSFCRSAT', 'PSFDEGRD')
metadata = {'filename': []}
for kw in header_keywords:
    metadata[kw] = []

In [5]:
for fn in glob.glob('./data/laplace/dd1/contemp_flats_repaired/*.fits'):
    metadata['filename'].append(fn)
    with open(fn, 'rb') as f:
        hdul = fits.open(f)
        for kw in header_keywords:
            metadata[kw].append(hdul[0].header[kw])

In [6]:
images = pd.DataFrame(metadata)
images

,APERTURE,DEC_TARG,FILTER,PA_V3,PSFCRSAT,PSFDEGRD,PSFQUAL,RA_TARG,TARGCNTR,TARGNAME,TARGTYPE,filename
0,NIC2-CORON,28.321111,F110W,245.023605,NO,NO,OK,64.669500,CENTERED,V892-TAU,POINT_SOURCE,./data/laplace/dd1/contemp_flats_repaired/n47f...
1,NIC2-CORON,28.321111,F160W,245.023605,NO,YES,BAD,64.669500,CENTERED,V892-TAU,POINT_SOURCE,./data/laplace/dd1/contemp_flats_repaired/n47f...
2,NIC2-CORON,28.321111,F187W,245.023605,NO,NO,OK,64.669500,CENTERED,V892-TAU,DOUBLE_STAR,./data/laplace/dd1/contemp_flats_repaired/n47f...
3,NIC2-CORON,28.321111,F205W,245.023605,NO,NO,OK,64.669500,CENTERED,V892-TAU,DOUBLE_STAR,./data/laplace/dd1/contemp_flats_repaired/n47f...
4,NIC2-CORON,26.180389,F110W,352.049988,NO,NO,OK,69.619042,CENTERED,DO-TAU,POINT_SOURCE,./data/laplace/dd1/contemp_flats_repaired/n47f...
5,NIC2-CORON,26.180389,F160W,352.049988,NO,YES,BAD,69.619042,CENTERED,DO-TAU,POINT_SOURCE,./data/laplace/dd1/contemp_flats_repaired/n47f...
6,NIC2-CORON,26.180389,F187W,352.049988,NO,NO,OK,69.619042,CENTERED,DO-TAU,POINT_SOURCE,./data/laplace/dd1/contemp_flats_repaired/n47f...
7,NIC2-CORON,26.180389,F205W,352.049988,NO,NO,OK,69.619042,CENTERED,DO-TAU,POINT_SOURCE,./data/laplace/dd1/contemp_flats_repaired/n47f...
8,NIC2-CORON,67.960547,F110W,320.314392,NO,NO,OK,311.474542,CENTERED,PV-CEP,EXTENDED_SOURCE,./data/laplace/dd1/contemp_flats_repaired/n47f...
9,NIC2-CORON,67.960547,F160W,320.314392,NO,YES,BAD,311.474542,CENTERED,PV-CEP,POINT_SOURCE,./data/laplace/dd1/contemp_flats_repaired/n47f...


In [7]:
quality_filter = (
    (images['FILTER'] == 'F160W') &
    (images['PSFQUAL'] == 'OK') &
    (images['PSFCRSAT'] == 'NO') &
    (images['TARGCNTR'] == 'CENTERED') &
    (images['APERTURE'] == 'NIC2-CORON')
)
images[quality_filter]

,APERTURE,DEC_TARG,FILTER,PA_V3,PSFCRSAT,PSFDEGRD,PSFQUAL,RA_TARG,TARGCNTR,TARGNAME,TARGTYPE,filename
12,NIC2-CORON,51.822222,F160W,331.156097,NO,NO,OK,255.353750,CENTERED,PG1700+518,EXTENDED_SOURCE,./data/laplace/dd1/contemp_flats_repaired/n4fh...
13,NIC2-CORON,51.822222,F160W,331.156097,NO,NO,OK,255.353750,CENTERED,PG1700+518,EXTENDED_SOURCE,./data/laplace/dd1/contemp_flats_repaired/n4fh...
14,NIC2-CORON,51.822222,F160W,331.156097,NO,NO,OK,255.353750,CENTERED,PG1700+518,EXTENDED_SOURCE,./data/laplace/dd1/contemp_flats_repaired/n4fh...
15,NIC2-CORON,51.822222,F160W,331.156097,NO,NO,OK,255.353750,CENTERED,PG1700+518,EXTENDED_SOURCE,./data/laplace/dd1/contemp_flats_repaired/n4fh...
16,NIC2-CORON,48.070000,F160W,321.297089,NO,NO,OK,259.909583,CENTERED,PG1718+4807,FAINT_OBJECT,./data/laplace/dd1/contemp_flats_repaired/n4fh...
17,NIC2-CORON,48.070000,F160W,321.297089,NO,NO,OK,259.909583,CENTERED,PG1718+4807,FAINT_OBJECT,./data/laplace/dd1/contemp_flats_repaired/n4fh...
18,NIC2-CORON,48.070000,F160W,321.297089,NO,NO,OK,259.909583,CENTERED,PG1718+4807,FAINT_OBJECT,./data/laplace/dd1/contemp_flats_repaired/n4fh...
19,NIC2-CORON,48.070000,F160W,321.297089,NO,NO,OK,259.909583,CENTERED,PG1718+4807,FAINT_OBJECT,./data/laplace/dd1/contemp_flats_repaired/n4fh...
20,NIC2-CORON,48.070000,F160W,321.297089,NO,NO,OK,259.909583,CENTERED,PG1718+4807,FAINT_OBJECT,./data/laplace/dd1/contemp_flats_repaired/n4fh...
21,NIC2-CORON,48.070000,F160W,321.297089,NO,NO,OK,259.909583,CENTERED,PG1718+4807,FAINT_OBJECT,./data/laplace/dd1/contemp_flats_repaired/n4fh...


In [8]:
psf_references_filter = (
    (images['FILTER'] == 'F160W') &
    (images['PSFQUAL'] == 'OK') &
    (images['PSFCRSAT'] == 'NO') &
    (images['TARGCNTR'] == 'CENTERED') &
    (images['TARGTYPE'] == 'POINT_SOURCE') &
    (images['APERTURE'] == 'NIC2-CORON')
)
images[psf_references_filter]

,APERTURE,DEC_TARG,FILTER,PA_V3,PSFCRSAT,PSFDEGRD,PSFQUAL,RA_TARG,TARGCNTR,TARGNAME,TARGTYPE,filename
33,NIC2-CORON,54.539056,F160W,173.163605,NO,NO,OK,177.868333,CENTERED,PG1148+5454-PSF,POINT_SOURCE,./data/laplace/dd1/contemp_flats_repaired/n4fh...
34,NIC2-CORON,54.539056,F160W,173.163605,NO,NO,OK,177.868333,CENTERED,PG1148+5454-PSF,POINT_SOURCE,./data/laplace/dd1/contemp_flats_repaired/n4fh...
35,NIC2-CORON,54.539056,F160W,173.163605,NO,NO,OK,177.868333,CENTERED,PG1148+5454-PSF,POINT_SOURCE,./data/laplace/dd1/contemp_flats_repaired/n4fh...
36,NIC2-CORON,54.539056,F160W,173.163605,NO,NO,OK,177.868333,CENTERED,PG1148+5454-PSF,POINT_SOURCE,./data/laplace/dd1/contemp_flats_repaired/n4fh...
84,NIC2-CORON,14.563548,F160W,0.023307,NO,NO,OK,3.126191,CENTERED,SAO91772,POINT_SOURCE,./data/laplace/dd1/contemp_flats_repaired/n4n4...
85,NIC2-CORON,14.563548,F160W,0.023307,NO,NO,OK,3.126191,CENTERED,SAO91772,POINT_SOURCE,./data/laplace/dd1/contemp_flats_repaired/n4n4...
86,NIC2-CORON,14.563548,F160W,0.023307,NO,NO,OK,3.126191,CENTERED,SAO91772,POINT_SOURCE,./data/laplace/dd1/contemp_flats_repaired/n4n4...
87,NIC2-CORON,14.563548,F160W,0.023307,NO,NO,OK,3.126191,CENTERED,SAO91772,POINT_SOURCE,./data/laplace/dd1/contemp_flats_repaired/n4n4...
106,NIC2-CORON,14.563550,F160W,44.999901,NO,NO,OK,3.126187,CENTERED,SAO91772,POINT_SOURCE,./data/laplace/dd1/contemp_flats_repaired/n4n4...
107,NIC2-CORON,14.563550,F160W,44.999901,NO,NO,OK,3.126187,CENTERED,SAO91772,POINT_SOURCE,./data/laplace/dd1/contemp_flats_repaired/n4n4...


In [9]:
import photutils

In [10]:
from astropy.nddata.utils import Cutout2D

In [11]:
BOX_SIZE = 80 # pixels

In [12]:
images.to_records().dtype

dtype((numpy.record, [('index', '<i8'), ('APERTURE', 'O'), ('DEC_TARG', '<f8'), ('FILTER', 'O'), ('PA_V3', '<f8'), ('PSFCRSAT', 'O'), ('PSFDEGRD', 'O'), ('PSFQUAL', 'O'), ('RA_TARG', '<f8'), ('TARGCNTR', 'O'), ('TARGNAME', 'O'), ('TARGTYPE', 'O'), ('filename', 'O')]))

In [13]:
def cutouts_cube_fits(dataframe):
    cutout_psfs = []
    for row in dataframe.itertuples():
        with open(row.filename, 'rb') as f:
            hdul = fits.open(f, memmap=False)
            x, y = hdul[0].header['TARSIAFX'], hdul[0].header['TARSIAFY']
            cutout = Cutout2D(hdul['SCI'].data, (x, y), BOX_SIZE)
            cutout_psfs.append(cutout.data)
    cutout_psfs_cube = np.asarray(cutout_psfs)
    images_recarray = dataframe.to_records()
    # Force strings to be strings, not objects
    images_recarray = images_recarray.astype(np.dtype((numpy.record, [('index', '<i8'), ('APERTURE', 'U20'), ('DEC_TARG', '<f8'), ('FILTER', 'U20'), ('PA_V3', '<f8'), ('PSFCRSAT', 'U20'), ('PSFDEGRD', 'U20'), ('PSFQUAL', 'U20'), ('RA_TARG', '<f8'), ('TARGCNTR', 'U20'), ('TARGNAME', 'U20'), ('TARGTYPE', 'U20'), ('filename', 'U80')])))
    table_hdu = fits.TableHDU(data=images_recarray)
    cube_hdul = fits.HDUList(
        [
            fits.PrimaryHDU(),
            fits.ImageHDU(cutout_psfs_cube),
            table_hdu
        ]
    )
    return cube_hdul

In [14]:
psf_ref_hdul = cutouts_cube_fits(images[psf_references_filter])
psf_ref_hdul.writeto('./data/laplace/nicmos_f160w_psf_ref.fits', overwrite=True)

I21
A20
D25.17
A20
D25.17
A20
A20
A20
D25.17
A20
A20
A20
A80


In [15]:
data_hdul = cutouts_cube_fits(images[quality_filter])
data_hdul.writeto('./data/laplace/nicmos_f160w_data.fits', overwrite=True)

I21
A20
D25.17
A20
D25.17
A20
A20
A20
D25.17
A20
A20
A20
A80


In [16]:
!du -hs ./data/laplace/nicmos_*.fits

 35M	./data/laplace/nicmos_f160w_data.fits
 27M	./data/laplace/nicmos_f160w_psf_ref.fits
